In [281]:
# Import packages

import pandas as pd 
import numpy as np 
import os

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

import xgboost as xgb

from joblib import dump, load

pd.set_option('display.max_columns', None)

In [282]:
stats = pd.read_csv('../../pipeline/data/07_adjusted_value_models_aggregated.csv')
schedule = pd.read_csv('../../data/schedule_final.csv')
betting = pd.read_csv('../../data/betting_data_cleaned_with_returns.csv')
betting = betting.drop(columns=['Unnamed: 0'])

stats = stats.dropna()

In [283]:
nfl = schedule.copy()
nfl = nfl.merge(stats.add_suffix('_home'), how='left', left_on = ['season', 'week', 'home', 'home_qb_abv'], 
                right_on = ['season_home', 'week_home', 'team_full_home', 'qb_home'])
nfl = nfl.merge(stats.add_suffix('_away'), how='left', left_on = ['season', 'week', 'away', 'away_qb_abv'], 
                right_on = ['season_away', 'week_away', 'team_full_away', 'qb_away'])
nfl = nfl[['date', 'season', 'week', 'season_type', 'home', 'away', 'home_score', 'away_score', 'home_qb', 'away_qb',
          'passing_value_adjusted_home', 'rushing_value_adjusted_home', 'pass_def_value_adjusted_home', 'rush_def_value_adjusted_home', 'special_teams_value_home',
          'passing_value_adjusted_away', 'rushing_value_adjusted_away', 'pass_def_value_adjusted_away', 'rush_def_value_adjusted_away', 'special_teams_value_away',
          'total_possession_time_standardized_home', 'total_possession_time_standardized_away', 
          'total_plays_standardized_home', 'total_plays_standardized_away',
          'pass_percentage_standardized_home', 'pass_percentage_standardized_away']]
nfl.head()

date  season  week season_type                 home  \
0  2014-09-04    2014     1         REG     Seattle Seahawks   
1  2014-09-07    2014     1         REG     Baltimore Ravens   
2  2014-09-07    2014     1         REG       Houston Texans   
3  2014-09-07    2014     1         REG        Chicago Bears   
4  2014-09-07    2014     1         REG  Pittsburgh Steelers   

                       away  home_score  away_score             home_qb  \
0         Green Bay Packers          36          16      Russell Wilson   
1        Cincinnati Bengals          16          23          Joe Flacco   
2  Washington Football Team          17           6    Ryan Fitzpatrick   
3             Buffalo Bills          20          23          Jay Cutler   
4          Cleveland Browns          30          27  Ben Roethlisberger   

          away_qb  passing_value_adjusted_home  rushing_value_adjusted_home  \
0   Aaron Rodgers                          NaN                          NaN   
1     Andy Dalton                          NaN                          NaN   
2  Robert Griffin                          NaN                          NaN   
3       EJ Manuel                          NaN                          NaN   
4     Brian Hoyer                          NaN                          NaN   

   pass_def_value_adjusted_home  rush_def_value_adjusted_home  \
0                           NaN                           NaN   
1                           NaN                           NaN   
2                           NaN                           NaN   
3                           NaN                           NaN   
4                           NaN                           NaN   

   special_teams_value_home  passing_value_adjusted_away  \
0                       NaN                          NaN   
1                       NaN                          NaN   
2                       NaN                          NaN   
3                       NaN                          NaN   
4                       NaN                          NaN   

   rushing_value_adjusted_away  pass_def_value_adjusted_away  \
0                          NaN                           NaN   
1                          NaN                           NaN   
2                          NaN                           NaN   
3                          NaN                           NaN   
4                          NaN                           NaN   

   rush_def_value_adjusted_away  special_teams_value_away  \
0                           NaN                       NaN   
1                           NaN                       NaN   
2                           NaN                       NaN   
3                           NaN                       NaN   
4                           NaN                       NaN   

   total_possession_time_standardized_home  \
0                                      NaN   
1                                      NaN   
2                                      NaN   
3                                      NaN   
4                                      NaN   

   total_possession_time_standardized_away  total_plays_standardized_home  \
0                                      NaN                            NaN   
1                                      NaN                            NaN   
2                                      NaN                            NaN   
3                                      NaN                            NaN   
4                                      NaN                            NaN   

   total_plays_standardized_away  pass_percentage_standardized_home  \
0                            NaN                                NaN   
1                            NaN                                NaN   
2                            NaN                                NaN   
3                            NaN                                NaN   
4                            NaN                                NaN   

   pass_percentage_standard

In [284]:
# Join in betting data to get point spreads

nfl_cleaned_with_betting = nfl.merge(betting, left_on=['date', 'home', 'away'],
                                            right_on=['date', 'home_team','away_team'])

In [285]:
# Remove any pushes, add binary label for home cover

nfl_cleaned_with_betting_final = nfl_cleaned_with_betting.copy()
nfl_cleaned_with_betting_final['home_cover'] = nfl_cleaned_with_betting_final.apply(
    lambda x: 1 if x.spread_cover_result == 'home' else 0, axis=1)
nfl_cleaned_with_betting_final = nfl_cleaned_with_betting_final[nfl_cleaned_with_betting_final.spread_cover_result != 'push']
nfl_cleaned_with_betting_final

date  season  week season_type                 home  \
0     2014-09-04    2014     1         REG     Seattle Seahawks   
1     2014-09-07    2014     1         REG     Baltimore Ravens   
2     2014-09-07    2014     1         REG       Houston Texans   
3     2014-09-07    2014     1         REG        Chicago Bears   
4     2014-09-07    2014     1         REG  Pittsburgh Steelers   
...          ...     ...   ...         ...                  ...   
2402  2023-01-22    2022    20        POST        Buffalo Bills   
2403  2023-01-22    2022    20        POST  San Francisco 49ers   
2404  2023-01-29    2022    21        POST  Philadelphia Eagles   
2405  2023-01-29    2022    21        POST   Kansas City Chiefs   
2406  2023-02-12    2022    22        POST  Philadelphia Eagles   

                          away  home_score_x  away_score_x  \
0            Green Bay Packers            36            16   
1           Cincinnati Bengals            16            23   
2     Washington Football Team            17             6   
3                Buffalo Bills            20            23   
4             Cleveland Browns            30            27   
...                        ...           ...           ...   
2402        Cincinnati Bengals            10            27   
2403            Dallas Cowboys            19            12   
2404       San Francisco 49ers            31             7   
2405        Cincinnati Bengals            23            20   
2406        Kansas City Chiefs            35            38   

                 home_qb          away_qb  passing_value_adjusted_home  \
0         Russell Wilson    Aaron Rodgers                          NaN   
1             Joe Flacco      Andy Dalton                          NaN   
2       Ryan Fitzpatrick   Robert Griffin                          NaN   
3             Jay Cutler        EJ Manuel                          NaN   
4     Ben Roethlisberger      Brian Hoyer                          NaN   
...                  ...              ...                          ...   
2402          Josh Allen       Joe Burrow                    -0.151187   
2403         Brock Purdy     Dak Prescott                     1.271880   
2404         Jalen Hurts      Brock Purdy                     0.131127   
2405     Patrick Mahomes       Joe Burrow                     0.474375   
2406         Jalen Hurts  Patrick Mahomes                    -0.017934   

      rushing_value_adjusted_home  pass_def_value_adjusted_home  \
0                             NaN                           NaN   
1                             NaN                           NaN   
2                             NaN                           NaN   
3                             NaN                           NaN   
4                             NaN                           NaN   
...                           ...                           ...   
2402                     0.650933                      0.483813   
2403                     0.634522                      0.303955   
2404                     1.616761                      0.742787   
2405                     0.512161                      0.781146   
2406                     1.316055                      0.526715   

      rush_def_value_adjusted_home  special_teams_value_home  \
0                              NaN                       NaN   
1                              NaN                       NaN   
2                              NaN                       NaN   
3                              NaN                       NaN   
4                              NaN                       NaN   
...                            ...                       ...   
2402                      0.589479                 -0.215688   
2403                      0.348498                  0.331705   
2404                     -0.131826                  0.329906   
2405                     -0.771093                 -0.504759   
2406                     -0.133076                  0.091893   


In [286]:
nfl_cleaned_with_betting_final.to_csv('../../data/adjusted_stats_point_spread_modeling_base_df.csv')

In [287]:
# Split data into train, validation, test sets

train_df = nfl_cleaned_with_betting_final[nfl_cleaned_with_betting_final.season <= 2020]
train_df = train_df.dropna()

val_df = nfl_cleaned_with_betting_final[nfl_cleaned_with_betting_final.season == 2021]
val_df = val_df.dropna()

test_df = nfl_cleaned_with_betting_final[nfl_cleaned_with_betting_final.season == 2022]
test_df = test_df.dropna()

In [288]:
# Feature Selection

feature_selection_list = [
    'passing_value_adjusted_home'
    , 'rushing_value_adjusted_home'
    , 'pass_def_value_adjusted_home'
    , 'rush_def_value_adjusted_home'
    , 'special_teams_value_home'
    , 'passing_value_adjusted_away'
    , 'rushing_value_adjusted_away'
    , 'pass_def_value_adjusted_away'
    , 'rush_def_value_adjusted_away'
    , 'special_teams_value_away'
    , 'total_possession_time_standardized_home'
    , 'total_possession_time_standardized_away'
    , 'total_plays_standardized_home'
    , 'total_plays_standardized_away'
    , 'pass_percentage_standardized_home'
    , 'pass_percentage_standardized_away'
    , 'home_spread'
    , 'home_cover'
    , 'total_score_line'
]
train_df[feature_selection_list].corr()

passing_value_adjusted_home  \
passing_value_adjusted_home                                 1.000000   
rushing_value_adjusted_home                                 0.156529   
pass_def_value_adjusted_home                                0.073299   
rush_def_value_adjusted_home                               -0.078702   
special_teams_value_home                                    0.042031   
passing_value_adjusted_away                                 0.019550   
rushing_value_adjusted_away                                 0.035015   
pass_def_value_adjusted_away                                0.031423   
rush_def_value_adjusted_away                               -0.034674   
special_teams_value_away                                   -0.004152   
total_possession_time_standardized_home                     0.385490   
total_possession_time_standardized_away                     0.003576   
total_plays_standardized_home                               0.208186   
total_plays_standardized_away                               0.038311   
pass_percentage_standardized_home                          -0.197499   
pass_percentage_standardized_away                          -0.005827   
home_spread                                                -0.375334   
home_cover                                                 -0.036804   
total_score_line                                            0.381442   

                                         rushing_value_adjusted_home  \
passing_value_adjusted_home                                 0.156529   
rushing_value_adjusted_home                                 1.000000   
pass_def_value_adjusted_home                                0.041304   
rush_def_value_adjusted_home                               -0.079103   
special_teams_value_home                                   -0.001642   
passing_value_adjusted_away                                -0.002672   
rushing_value_adjusted_away                                -0.034786   
pass_def_value_adjusted_away                                0.031916   
rush_def_value_adjusted_away                                0.006557   
special_teams_value_away                                    0.047439   
total_possession_time_standardized_home                     0.177246   
total_possession_time_standardized_away                     0.020678   
total_plays_standardized_home                               0.068509   
total_plays_standardized_away                               0.007773   
pass_percentage_standardized_home                          -0.361668   
pass_percentage_standardized_away                          -0.006779   
home_spread                                                -0.232185   
home_cover                                                  0.033367   
total_score_line                                            0.165522   

                                         pass_def_value_adjusted_home  \
passing_value_adjusted_home                                  0.073299   
rushing_value_adjusted_home                                  0.041304   
pass_def_value_adjusted_home                                 1.000000   
rush_def_value_adjusted_home                                 0.087944   
special_teams_value_home                                    -0.014403   
passing_value_adjusted_away                                  0.011351   
rushing_value_adjusted_away                                  0.011494   
pass_def_value_adjusted_away                                 0.015483   
rush_def_value_adjusted_away                                 0.001829   
special_teams_value_away                                    -0.016879   
total_possession_time_standardized_home                      0.290015   
total_possession_time_standardized_away                     -0.014612   
total_plays_standardized_home                                0.147254   
total_plays_standardized_away                               -0.049080   
pass_percentage_standardized_home                           -0.3

In [290]:
feature_list = [
    'passing_value_adjusted_home'
    , 'rushing_value_adjusted_home'
    , 'pass_def_value_adjusted_home'
    , 'rush_def_value_adjusted_home'
    , 'special_teams_value_home'
    , 'passing_value_adjusted_away'
    , 'rushing_value_adjusted_away'
    , 'pass_def_value_adjusted_away'
    , 'rush_def_value_adjusted_away'
    , 'special_teams_value_away'
#     , 'total_possession_time_standardized_home'
#     , 'total_possession_time_standardized_away'
#     , 'total_plays_standardized_home'
#     , 'total_plays_standardized_away'
#     , 'pass_percentage_standardized_home'
#     , 'pass_percentage_standardized_away'
    , 'home_spread'
]

In [291]:
# Get features, labels for train, val, and test sets

train_x = train_df[feature_list].to_numpy()
train_y = train_df.home_cover.to_numpy()

val_x = val_df[feature_list].to_numpy()
val_y = val_df.home_cover.to_numpy()

test_x = test_df[feature_list].to_numpy()
test_y = test_df.home_cover.to_numpy()

In [292]:
# Logistic Regression Modeling

model = LogisticRegression().fit(train_x, train_y)

train_preds = model.predict(train_x)
train_probs = model.predict_proba(train_x)

val_preds = model.predict(val_x)
val_probs = model.predict_proba(val_x)

print('Training Accuracy: {}'.format(np.round(accuracy_score(train_preds, train_y), 3)))
print('Validation Accuracy: {}'.format(np.round(accuracy_score(val_preds, val_y), 3)))

Training Accuracy: 0.56
Validation Accuracy: 0.547


In [293]:
# Logistic Regression, with hyperparameter tuning

def logistic_regression_tuning(train_x, train_y, verbose=True):
    
    # Hyperparameters to tune
    penalty_list = ['none', 'l2', 'l1']
    c_values = [100, 10, 1.0, 0.1, 0.01]
    solvers = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
    
    train_acc_chosen_model = 0
    best_val_acc = 0
    
    for penalty in penalty_list:
        for c in c_values:
            for solver in solvers:
                
                if solver == 'newton-cg' and penalty in ['l1', 'elasticnet']:
                    pass
                elif solver == 'liblinear' and penalty in ['none', 'elasticnet']:
                    pass
                elif solver == 'lbfgs' and penalty in ['l1', 'elasticnet']:
                    pass
                elif solver == 'sag' and penalty in ['l1', 'elasticnet']:
                    pass
                
                else:
                
                    lr_model = LogisticRegression(penalty=penalty, solver=solver, C=c).fit(train_x, train_y)
                    train_acc = accuracy_score(lr_model.predict(train_x), train_y)
                    val_acc = accuracy_score(lr_model.predict(val_x), val_y)

                    if verbose:
                        print('Train Accuracy: {}, Validation Accuracy:{}'.format(train_acc, val_acc))

                    if val_acc > best_val_acc:
                        best_val_acc = val_acc
                        train_acc_chosen_model = train_acc
                        best_params = [penalty, c, solver]
                    
    print('Best Model:')
    print('Training Accuracy: {}'.format(train_acc_chosen_model))
    print('Validation Accuracy: {}'.format(best_val_acc))
    print('Best Parameters: {}'.format(best_params))
    
    return best_params

In [294]:
best_params_lr = logistic_regression_tuning(train_x, train_y, verbose = False)

/home/ian/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/home/ian/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/home/ian/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/home/ian/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/home/ian/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be re

Best Model:
Training Accuracy: 0.5251469627694317
Validation Accuracy: 0.5510204081632653
Best Parameters: ['l1', 0.1, 'saga']


In [295]:
# Save best logistic regression model 

best_penalty = best_params_lr[0]
best_c = best_params_lr[1]
best_solver = best_params_lr[2]

best_lr_model = LogisticRegression(penalty=best_penalty, C=best_c, solver=best_solver).fit(train_x, train_y)
best_lr_model = LogisticRegression().fit(train_x, train_y)
dump(best_lr_model, 'saved_models/ps_logistic_regression_av.joblib')

['saved_models/ps_logistic_regression_av.joblib']

In [296]:
# Random Forest Modeling

model = RandomForestClassifier().fit(train_x, train_y)

train_preds = model.predict(train_x)
train_probs = model.predict_proba(train_x)

val_preds = model.predict(val_x)
val_probs = model.predict_proba(val_x)

print('Training Accuracy: {}'.format(np.round(accuracy_score(train_preds, train_y), 3)))
print('Validation Accuracy: {}'.format(np.round(accuracy_score(val_preds, val_y), 3)))

Training Accuracy: 1.0
Validation Accuracy: 0.527


In [297]:
# Random Forest, with hyperparameter tuning

def random_forest_tuning(train_x, train_y, verbose=True):
    
    # Hyperparameters to tune
#     estimators_list = [150, 200, 300, 400, 500, 1000]
    estimators_list = [50, 100, 200, 500]
    criterions = ['gini', 'entropy', 'log_loss']
    depths = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, None]
#     depths = [5, 6, 7, 8, 9, 10, None]
    
    train_acc_chosen_model = 0
    best_val_acc = 0
    
    for n_estimators in estimators_list:
        for criterion in criterions:
            for depth in depths:
                
                if False:
                    pass
                
                else:
                
                    rf_model = RandomForestClassifier(
                        n_estimators=n_estimators, criterion=criterion, max_depth=depth).fit(train_x, train_y)
                    train_acc = accuracy_score(rf_model.predict(train_x), train_y)
                    val_acc = accuracy_score(rf_model.predict(val_x), val_y)

                    if verbose:
                        print('Train Accuracy: {}, Validation Accuracy:{}, Params: {}'.format(
                            train_acc, val_acc, [n_estimators, criterion, depth]))

                    if val_acc > best_val_acc:
                        best_val_acc = val_acc
                        train_acc_chosen_model = train_acc
                        best_params = [n_estimators, criterion, depth]
                        best_model = rf_model
                    
    print('Best Model:')
    print('Training Accuracy: {}'.format(train_acc_chosen_model))
    print('Validation Accuracy: {}'.format(best_val_acc))
    print('Best Parameters: {}'.format(best_params))
    
    return best_params, best_model

In [298]:
best_params_rf, best_rf_model = random_forest_tuning(train_x, train_y, verbose = True)

Train Accuracy: 0.5453951665578054, Validation Accuracy:0.5224489795918368, Params: [50, 'gini', 1]
Train Accuracy: 0.6002612671456564, Validation Accuracy:0.5265306122448979, Params: [50, 'gini', 2]
Train Accuracy: 0.6329196603527106, Validation Accuracy:0.5183673469387755, Params: [50, 'gini', 3]
Train Accuracy: 0.6903984323971261, Validation Accuracy:0.5102040816326531, Params: [50, 'gini', 4]
Train Accuracy: 0.7491835401698237, Validation Accuracy:0.5224489795918368, Params: [50, 'gini', 5]
Train Accuracy: 0.8131939908556499, Validation Accuracy:0.5142857142857142, Params: [50, 'gini', 6]
Train Accuracy: 0.8634879163945134, Validation Accuracy:0.5428571428571428, Params: [50, 'gini', 7]
Train Accuracy: 0.9229261920313521, Validation Accuracy:0.49795918367346936, Params: [50, 'gini', 8]
Train Accuracy: 0.9451338994121489, Validation Accuracy:0.5020408163265306, Params: [50, 'gini', 9]
Train Accuracy: 0.9634225996080993, Validation Accuracy:0.5591836734693878, Params: [50, 'gini', 10

Train Accuracy: 0.7328543435662965, Validation Accuracy:0.5102040816326531, Params: [200, 'entropy', 5]
Train Accuracy: 0.8131939908556499, Validation Accuracy:0.5102040816326531, Params: [200, 'entropy', 6]
Train Accuracy: 0.8706727629000653, Validation Accuracy:0.5183673469387755, Params: [200, 'entropy', 7]
Train Accuracy: 0.9085564990202482, Validation Accuracy:0.5265306122448979, Params: [200, 'entropy', 8]
Train Accuracy: 0.9366427171783148, Validation Accuracy:0.5061224489795918, Params: [200, 'entropy', 9]
Train Accuracy: 0.9588504245591117, Validation Accuracy:0.5428571428571428, Params: [200, 'entropy', 10]
Train Accuracy: 1.0, Validation Accuracy:0.5061224489795918, Params: [200, 'entropy', None]
Train Accuracy: 0.5382103200522534, Validation Accuracy:0.5224489795918368, Params: [200, 'log_loss', 1]
Train Accuracy: 0.5826257348138472, Validation Accuracy:0.5061224489795918, Params: [200, 'log_loss', 2]
Train Accuracy: 0.6355323318092749, Validation Accuracy:0.493877551020408

In [299]:
# Save best random forest model 

best_n_estimators = best_params_lr[0]
best_criterion = best_params_lr[1]
best_depth = best_params_lr[2]

dump(best_rf_model, 'saved_models/ps_random_forest_av.joblib')

['saved_models/ps_random_forest_av.joblib']

In [300]:
sum(best_rf_model.predict(test_x))

77

In [301]:
best_rf_model.n_estimators

50